In [2]:
import os
import fitz
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    from tqdm.auto import tqdm


# Step 0: Load environment and set paths
load_dotenv()

# Detect root folder (works even if running from /notebooks)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()

DATA_DIR = os.path.join(ROOT_DIR, "data")
CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")
EMBED_MODEL = "BAAI/bge-large-en-v1.5"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHROMA_DIR, exist_ok=True)

print(f"📂 Root dir: {ROOT_DIR}")
print(f"📁 Data dir: {DATA_DIR}")
print(f"💾 Chroma dir: {CHROMA_DIR}")

# Step 1: Load text from PDFs
def load_pdfs_from_folder(folder_path):
    all_docs = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            pdf = fitz.open(file_path)
            for page_num, page in enumerate(pdf, start=1):
                text = page.get_text("text") or ""
                if text.strip():
                    all_docs.append(
                        Document(
                            page_content=text.strip(),
                            metadata={"source": file, "page": page_num}
                        )
                    )
            pdf.close()
    return all_docs

docs = load_pdfs_from_folder(DATA_DIR)
print(f"✅ Loaded {len(docs)} pages from PDFs.")

if not docs:
    raise ValueError("❌ No PDF pages found! Please add PDFs inside the /data folder.")

# Step 2: Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"✅ Split into {len(chunks)} chunks.")

# Step 3: Create or update vector store
print("⚙️ Generating embeddings... (this might take a while)")
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

if os.listdir(CHROMA_DIR):
    print("📡 Found existing Chroma DB — updating it...")
    db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
    db.add_documents(chunks)
else:
    print("🆕 No Chroma DB found — creating a new one...")
    db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=CHROMA_DIR)

db.persist()
print(f"🎉 Vector store saved at: {CHROMA_DIR}")

# Step 4: Preview a few samples
sample = db._collection.get(limit=3)
print("🔍 Sample metadata:", sample["metadatas"])


📂 Root dir: d:\MyPocketLawyer-AI-Powered-Legal-Aid-Assistant
📁 Data dir: d:\MyPocketLawyer-AI-Powered-Legal-Aid-Assistant\data
💾 Chroma dir: d:\MyPocketLawyer-AI-Powered-Legal-Aid-Assistant\chroma_db
✅ Loaded 484 pages from PDFs.
✅ Split into 681 chunks.
⚙️ Generating embeddings... (this might take a while)
📡 Found existing Chroma DB — updating it...
🎉 Vector store saved at: d:\MyPocketLawyer-AI-Powered-Legal-Aid-Assistant\chroma_db
🔍 Sample metadata: [{'source': 'Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf', 'page': 1}, {'source': 'Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf', 'page': 2}, {'page': 3, 'source': 'Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf'}]


In [8]:
import os
import fitz
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

# Step 0: Load environment and set paths
load_dotenv()

# Detect root folder (works even if running from /notebooks)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()

DATA_DIR = os.path.join(ROOT_DIR, "data")
CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")
EMBED_MODEL = "BAAI/bge-large-en-v1.5"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHROMA_DIR, exist_ok=True)

print(f"📂 Root dir: {ROOT_DIR}")
print(f"📁 Data dir: {DATA_DIR}")
print(f"💾 Chroma dir: {CHROMA_DIR}")

📂 Root dir: /home/jagannath/myPocketLawyer
📁 Data dir: /home/jagannath/myPocketLawyer/data
💾 Chroma dir: /home/jagannath/myPocketLawyer/chroma_db


In [9]:
# Step 1: Load text from PDFs
def load_pdfs_from_folder(folder_path):
    all_docs = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            pdf = fitz.open(file_path)
            for page_num, page in enumerate(pdf, start=1):
                text = page.get_text("text") or ""
                if text.strip():
                    all_docs.append(
                        Document(
                            page_content=text.strip(),
                            metadata={"source": file, "page": page_num}
                        )
                    )
            pdf.close()
    return all_docs

docs = load_pdfs_from_folder(DATA_DIR)
print(f"✅ Loaded {len(docs)} pages from PDFs.")

if not docs:
    raise ValueError("❌ No PDF pages found! Please add PDFs inside the /data folder.")

✅ Loaded 484 pages from PDFs.


In [10]:
# Step 2: Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"✅ Split into {len(chunks)} chunks.")

✅ Split into 681 chunks.


In [13]:
# Step 3: Create or update vector store
print("⚙️ Generating embeddings... (this might take a while)")
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

if os.listdir(CHROMA_DIR):
    print("📡 Found existing Chroma DB — updating it...")
    db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
    db.add_documents(chunks)
else:
    print("🆕 No Chroma DB found — creating a new one...")
    db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=CHROMA_DIR)

db.persist()
print(f"🎉 Vector store saved at: {CHROMA_DIR}")

⚙️ Generating embeddings... (this might take a while)
🆕 No Chroma DB found — creating a new one...
🎉 Vector store saved at: /home/jagannath/myPocketLawyer/chroma_db


/tmp/ipykernel_38461/2651287052.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [14]:
# Step 4: Preview a few samples
sample = db._collection.get(limit=3)
print("🔍 Sample metadata:", sample["metadatas"])

🔍 Sample metadata: [{'source': 'criminal-code-nepal.pdf', 'page': 1}, {'page': 2, 'source': 'criminal-code-nepal.pdf'}, {'page': 3, 'source': 'criminal-code-nepal.pdf'}]
